In [1]:
%%capture
!unzip Project3Data.zip

In [2]:
import pandas as pd

# Look at a sample clinical note
data = pd.read_xml('Project3Data/Project3_data/400-01.xml')

data['TEXT'][0]

'Record date: 2062-06-10\n\nMedical Walk In Unit\n\nHartsville General Hospital\n\n21 Brook St \n\nPataskala, RI 33689\n\nPhone: 929-462-2794   Fax: 648-875-5821 \n\n\n\nPCP: \n\n\n\n\n\nNursing Assessment:  \n\nWT:    Temp:    P: 100     BP: 190/116\n\n\n\nReason for visit: Pt just left the EW.  Noted HTN has been off meds for a while.  Was nor given any meds\n\n\n\nLab Data :\n\n\n\nAllergies:      \n\n\n\n\n\nMedication: \n\noff meds for more than 3 months\n\n\n\nHistory of the Present Illness:                                \n\n59 y/o female here for high blood pressure.  Pt was seen in EW for medications and was given Seroquel, Neurontin, and amytriptyline.  Pt has been out of meds for more than 3 months.\n\n\n\nPt remembers being atenolol and HCTZ.  Pt plans to come to the medical walk-in tomorrow with all her medications.\n\n\n\nPt denies HA, visual problems, chest pain, problems with urination.\n\n\n\nFather had CAD and died at age 55.  Mother died when she was 89 (10 months ag

The goal is to provide a frequency distribution of the vital signs/physical exam readings observed across the patient clinical notes.

What are the vital signs/physical exams?
*   Blood pressure
*   Pulse
*   Temperature
*   Respitory rate
*   Height
*   Weight
*   Oxygen saturation

Since we are analyzing free-text data that is only semi-structured, I read through a bunch of clinical notes and manually extracted aliases for each vital sign.
*   Blood pressure: ```[Blood pressure, BP]```
*   Pulse: ```[Pulse, P, HR, heart rate]```
*   Temperature: ```[Tempreture, T, Temp]```
*   Respitory rate: ```[Respitory rate, Respirations, RR, Fs]```
*   Height: ```[Height, Ht]```
*   Weight: ```[Wt, wgt, none (only a number followed by 'pounds' or 'lbs'), weighing, W.]```
*   Oxygen saturation: ```[Oxygen saturation, SaO2, O2 saturation, saturating, Sat, O2 sat, O2sat]```

In [3]:
# Extract the name of all the files to be analyzed
file_names = []
with open('Project3Data/Project3_data/MIscellaneous/pl.txt', 'r') as f:
  for file_name in f:
    file_names.append(file_name.rstrip('\n'))
f.close()
file_names[0]

'100-01.xml'

In [24]:
# Define the aliases and regex patterns for each vital sign
patterns = {
    'Blood pressure': r'\b(?:blood pressure|bp)\b(?:(?!\b(?:blood pressure|bp|pulse|p\b|hr|heart rate|temperature|t\b|temp|respiratory rate|respirations|rr|fs|height|ht|wt|weight|weighing|w\.|oxygen saturation|sao2|o2 saturation|saturating|sat|o2 sat)\b).)*?(\d+/\d+)',
    'Pulse': r'\b(?:pulse|p\b|hr|heart rate)\b\s*[:\-]?\s*(\d+)',
    'Temperature': r'\b(?:temperature|t\b|temp)\b(?:(?!\b(?:blood pressure|bp|pulse|p\b|hr|heart rate|temperature|t\b|temp|respiratory rate|respirations|rr|fs|height|ht|wt|weight|weighing|w\.|oxygen saturation|sao2|o2 saturation|saturating|sat|o2 sat)\b).)*?(\d+(?:\.\d+)?)',
    'Respiratory rate': r'\b(?:respiratory rate|respirations|rr|fs)\b(?:(?!\b(?:blood pressure|bp|pulse|p\b|hr|heart rate|temperature|t\b|temp|respiratory rate|respirations|rr|fs|height|ht|wt|weight|weighing|w\.|oxygen saturation|sao2|o2 saturation|saturating|sat|o2 sat)\b).)*?(\d+)',
    'Height': r'\b(?:height|ht)\b(?:(?!\b(?:blood pressure|bp|pulse|p\b|hr|heart rate|temperature|t\b|temp|respiratory rate|respirations|rr|fs|height|ht|wt|weight|weighing|w\.|oxygen saturation|sao2|o2 saturation|saturating|sat|o2 sat)\b).)*?(\d+)',
    'Weight': r'\b(?:wt|wgt|weight|weighing|w\.)\b(?:(?!\b(?:blood pressure|bp|pulse|p\b|hr|heart rate|temperature|t\b|temp|respiratory rate|respirations|rr|fs|height|ht|wt|weight|weighing|w\.|oxygen saturation|sao2|o2 saturation|saturating|sat|o2 sat)\b).)*?(\d+)',
    'Oxygen saturation': r'\b(?:oxygen saturation|sao2|o2 saturation|saturating|sat|o2 sat|o2sat)\b(?:(?!\b(?:blood pressure|bp|pulse|p\b|hr|heart rate|temperature|t\b|temp|respiratory rate|respirations|rr|fs|height|ht|wt|weight|weighing|w\.|oxygen saturation|sao2|o2 saturation|saturating|sat|o2 sat)\b).)*?(\d+%)'
}

In [25]:
# Define the valid ranges for each vital sign
valid_ranges = {
    'Blood pressure': lambda x: 80 <= int(x.split('/')[0]) <= 180 and 40 <= int(x.split('/')[1]) <= 110,
    'Pulse': lambda x: 40 <= int(x) <= 120,
    'Temperature': lambda x: 90 <= float(x) <= 110,
    'Respiratory rate': lambda x: 8 <= int(x) <= 30,
    'Height': lambda x: 0 <= int(x) <= 203,
    'Weight': lambda x: 100 <= int(x) <= 400,
    'Oxygen saturation': lambda x: 85 <= int(x.rstrip('%')) <= 100
}

In [27]:
import re

for xml_file in file_names[::50]:
  clinical_note = pd.read_xml(f'Project3Data/Project3_data/{xml_file}')['TEXT'][0]
  print(f'For {xml_file}:')
  for sign, pattern in patterns.items():
    matches = re.findall(pattern, clinical_note, re.IGNORECASE)
    valid_matches = [match for match in matches if valid_ranges[sign](match)]
    print(f'{sign}: {valid_matches}')
  print()

For 100-01.xml:
Blood pressure: []
Pulse: []
Temperature: []
Respiratory rate: []
Height: []
Weight: []
Oxygen saturation: []

For 111-04.xml:
Blood pressure: []
Pulse: []
Temperature: ['97.4']
Respiratory rate: ['16']
Height: []
Weight: []
Oxygen saturation: ['97%']

For 123-01.xml:
Blood pressure: ['130/90']
Pulse: ['64']
Temperature: []
Respiratory rate: []
Height: []
Weight: ['170']
Oxygen saturation: []

For 134-03.xml:
Blood pressure: ['136/70']
Pulse: ['72']
Temperature: []
Respiratory rate: []
Height: []
Weight: ['174']
Oxygen saturation: []

For 146-03.xml:
Blood pressure: []
Pulse: ['65']
Temperature: []
Respiratory rate: ['14']
Height: []
Weight: []
Oxygen saturation: []

For 157-04.xml:
Blood pressure: ['144/78']
Pulse: []
Temperature: []
Respiratory rate: []
Height: []
Weight: ['366']
Oxygen saturation: []

For 168-03.xml:
Blood pressure: ['135/80']
Pulse: ['78']
Temperature: ['97.4']
Respiratory rate: ['18']
Height: []
Weight: []
Oxygen saturation: ['99%']

For 179-02.xml